In [1]:
#Import requests library and API key
import requests
from config import weather_api_key

# Import Dependencies
import pandas as pd

### Import & edit necessary DataFrames for API call

In [2]:
# Import DataFrame from Segment One
initial_2019 = pd.read_csv('Database/Data/jan_19_clean_data.csv')
initial_2020 = pd.read_csv('Database/Data/jan_20_clean_data.csv')
initial_2019

,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,1,2,20363,9E,N8688C,3280,11953,1195302,GNV,10397,1039707,ATL,601.0,0.0,0600-0659,722.0,0.0,0,0,300
1,1,2,20363,9E,N348PQ,3281,13487,1348702,MSP,11193,1119302,CVG,1359.0,0.0,1400-1459,1633.0,0.0,0,0,596
2,1,2,20363,9E,N8896A,3282,11433,1143302,DTW,11193,1119302,CVG,1215.0,0.0,1200-1259,1329.0,0.0,0,0,229
3,1,2,20363,9E,N8886A,3283,15249,1524906,TLH,10397,1039707,ATL,1521.0,0.0,1500-1559,1625.0,0.0,0,0,223
4,1,2,20363,9E,N8974C,3284,10397,1039707,ATL,11778,1177801,FSM,1847.0,0.0,1900-1959,1940.0,0.0,0,0,579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565958,31,4,19977,UA,N776UA,200,12016,1201602,GUM,12173,1217305,HNL,749.0,0.0,0700-0759,1832.0,0.0,0,0,3801
565959,31,4,19977,UA,N36280,174,12016,1201602,GUM,14955,1495503,SPN,717.0,0.0,0700-0759,759.0,0.0,0,0,129
565960,31,4,19977,UA,N36280,117,14955,1495503,SPN,12016,1201602,GUM,857.0,0.0,0900-0959,933.0,0.0,0,0,129
565961,31,4,19977,UA,N39726,105,14955,1495503,SPN,12016,1201602,GUM,1820.0,0.0,1800-1859,1854.0,0.0,0,0,129


In [3]:
# Import City Codes to Cities DataFrame
city_code_df = pd.read_csv('Flight_data_files/airport_codes_city.csv')
city_code_df

,Airport Code,City
0,ABR,"Aberdeen, SD"
1,ABI,"Abilene, TX"
2,ADK,"Adak Island, AK"
3,CAK,"Akron/Canton, OH"
4,ALB,"Albany, NY"
...,...,...
317,ILM,"Wilmington, NC"
318,ORH,"Worcester, MA"
319,WRG,"Wrangell, AK"
320,YAK,"Yakutat, AK"


In [4]:
# Make all cities in lowercase
city_code_lower = city_code_df['City'].str.lower()
city_code_lower

0          aberdeen, sd
1           abilene, tx
2       adak island, ak
3      akron/canton, oh
4            albany, ny
             ...       
317      wilmington, nc
318       worcester, ma
319        wrangell, ak
320         yakutat, ak
321            yuma, az
Name: City, Length: 322, dtype: object

In [5]:
# Delete all spaces
city_code_clean = city_code_lower.str.replace(" ", "")
city_code_clean

0          aberdeen,sd
1           abilene,tx
2        adakisland,ak
3      akron/canton,oh
4            albany,ny
            ...       
317      wilmington,nc
318       worcester,ma
319        wrangell,ak
320         yakutat,ak
321            yuma,az
Name: City, Length: 322, dtype: object

In [6]:
# Convert the lowercase/no space city series to a dataframe
lower_city_df = city_code_clean.to_frame()
lower_city_df

,City
0,"aberdeen,sd"
1,"abilene,tx"
2,"adakisland,ak"
3,"akron/canton,oh"
4,"albany,ny"
...,...
317,"wilmington,nc"
318,"worcester,ma"
319,"wrangell,ak"
320,"yakutat,ak"


In [7]:
# Create a city abreviation dataframe
city_code_abr_df = city_code_df.drop(columns="City")
city_code_abr_df

,Airport Code
0,ABR
1,ABI
2,ADK
3,CAK
4,ALB
...,...
317,ILM
318,ORH
319,WRG
320,YAK


In [8]:
# Inner join the two dataframes to create a clean city code df that can be used by the API
city_code_clean_df = pd.concat([city_code_abr_df, lower_city_df], axis=1, join='inner')
city_code_clean_df

,Airport Code,City
0,ABR,"aberdeen,sd"
1,ABI,"abilene,tx"
2,ADK,"adakisland,ak"
3,CAK,"akron/canton,oh"
4,ALB,"albany,ny"
...,...,...
317,ILM,"wilmington,nc"
318,ORH,"worcester,ma"
319,WRG,"wrangell,ak"
320,YAK,"yakutat,ak"


In [9]:
# Create a dataframe for the origin cities
origin_code_df = city_code_clean_df.rename(columns={"Airport Code": "ORIGIN"})
origin_code_df

,ORIGIN,City
0,ABR,"aberdeen,sd"
1,ABI,"abilene,tx"
2,ADK,"adakisland,ak"
3,CAK,"akron/canton,oh"
4,ALB,"albany,ny"
...,...,...
317,ILM,"wilmington,nc"
318,ORH,"worcester,ma"
319,WRG,"wrangell,ak"
320,YAK,"yakutat,ak"


In [10]:
# Create a dataframe for the destination cities
dest_code_df = city_code_clean_df.rename(columns={"Airport Code": "DEST"})
dest_code_df

,DEST,City
0,ABR,"aberdeen,sd"
1,ABI,"abilene,tx"
2,ADK,"adakisland,ak"
3,CAK,"akron/canton,oh"
4,ALB,"albany,ny"
...,...,...
317,ILM,"wilmington,nc"
318,ORH,"worcester,ma"
319,WRG,"wrangell,ak"
320,YAK,"yakutat,ak"


In [11]:
initial_2019["dest_city"] = ""
initial_2019["origin_city"] = ""
flights_2019 = initial_2019[["DAY_OF_MONTH","DAY_OF_WEEK","OP_CARRIER_AIRLINE_ID","OP_CARRIER","TAIL_NUM","OP_CARRIER_FL_NUM","ORIGIN_AIRPORT_ID","ORIGIN_AIRPORT_SEQ_ID","ORIGIN","origin_city","DEST_AIRPORT_ID","DEST_AIRPORT_SEQ_ID","DEST","dest_city","DEP_TIME","DEP_DEL15","DEP_TIME_BLK","ARR_TIME","ARR_DEL15","CANCELLED","DIVERTED","DISTANCE"]]
flights_2019

,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,origin_city,...,DEST,dest_city,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,1,2,20363,9E,N8688C,3280,11953,1195302,GNV,,...,ATL,,601.0,0.0,0600-0659,722.0,0.0,0,0,300
1,1,2,20363,9E,N348PQ,3281,13487,1348702,MSP,,...,CVG,,1359.0,0.0,1400-1459,1633.0,0.0,0,0,596
2,1,2,20363,9E,N8896A,3282,11433,1143302,DTW,,...,CVG,,1215.0,0.0,1200-1259,1329.0,0.0,0,0,229
3,1,2,20363,9E,N8886A,3283,15249,1524906,TLH,,...,ATL,,1521.0,0.0,1500-1559,1625.0,0.0,0,0,223
4,1,2,20363,9E,N8974C,3284,10397,1039707,ATL,,...,FSM,,1847.0,0.0,1900-1959,1940.0,0.0,0,0,579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565958,31,4,19977,UA,N776UA,200,12016,1201602,GUM,,...,HNL,,749.0,0.0,0700-0759,1832.0,0.0,0,0,3801
565959,31,4,19977,UA,N36280,174,12016,1201602,GUM,,...,SPN,,717.0,0.0,0700-0759,759.0,0.0,0,0,129
565960,31,4,19977,UA,N36280,117,14955,1495503,SPN,,...,GUM,,857.0,0.0,0900-0959,933.0,0.0,0,0,129
565961,31,4,19977,UA,N39726,105,14955,1495503,SPN,,...,GUM,,1820.0,0.0,1800-1859,1854.0,0.0,0,0,129


In [12]:
# Merge Origin code and the flights dataframes
origin_join_df = pd.merge(origin_code_df,flights_2019, on='ORIGIN', how ='inner')
origin_join_df

,ORIGIN,City,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST,dest_city,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,2,20304,OO,N910EV,7363,10141,1014106,...,MSP,,1411.0,1.0,1300-1359,1521.0,1.0,0,0,257
1,ABR,"aberdeen,sd",1,2,20304,OO,N432SW,7365,10141,1014106,...,MSP,,642.0,0.0,0600-0659,800.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,3,20304,OO,N498CA,7363,10141,1014106,...,MSP,,1310.0,0.0,1300-1359,1424.0,0.0,0,0,257
3,ABR,"aberdeen,sd",2,3,20304,OO,N439SW,7365,10141,1014106,...,MSP,,502.0,0.0,0001-0559,622.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,4,20304,OO,N8965E,7363,10141,1014106,...,MSP,,1307.0,0.0,1300-1359,1411.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555444,YUM,"yuma,az",30,3,20304,OO,N746SK,2988,16218,1621802,...,PHX,,1739.0,0.0,1700-1759,1832.0,0.0,0,0,160
555445,YUM,"yuma,az",31,4,20304,OO,N713SK,2978,16218,1621802,...,PHX,,1120.0,0.0,1100-1159,1212.0,0.0,0,0,160
555446,YUM,"yuma,az",31,4,20304,OO,N713SK,2988,16218,1621802,...,PHX,,1739.0,0.0,1700-1759,1828.0,0.0,0,0,160
555447,YUM,"yuma,az",31,4,20304,OO,N710SK,3060,16218,1621802,...,PHX,,2120.0,0.0,2100-2159,2203.0,0.0,0,0,160


In [13]:
# Change city column to Origin City
origin_join_df = origin_join_df.rename(columns={'City':'ORIGIN_CITY'})
origin_join_df

,ORIGIN,ORIGIN_CITY,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST,dest_city,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,2,20304,OO,N910EV,7363,10141,1014106,...,MSP,,1411.0,1.0,1300-1359,1521.0,1.0,0,0,257
1,ABR,"aberdeen,sd",1,2,20304,OO,N432SW,7365,10141,1014106,...,MSP,,642.0,0.0,0600-0659,800.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,3,20304,OO,N498CA,7363,10141,1014106,...,MSP,,1310.0,0.0,1300-1359,1424.0,0.0,0,0,257
3,ABR,"aberdeen,sd",2,3,20304,OO,N439SW,7365,10141,1014106,...,MSP,,502.0,0.0,0001-0559,622.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,4,20304,OO,N8965E,7363,10141,1014106,...,MSP,,1307.0,0.0,1300-1359,1411.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555444,YUM,"yuma,az",30,3,20304,OO,N746SK,2988,16218,1621802,...,PHX,,1739.0,0.0,1700-1759,1832.0,0.0,0,0,160
555445,YUM,"yuma,az",31,4,20304,OO,N713SK,2978,16218,1621802,...,PHX,,1120.0,0.0,1100-1159,1212.0,0.0,0,0,160
555446,YUM,"yuma,az",31,4,20304,OO,N713SK,2988,16218,1621802,...,PHX,,1739.0,0.0,1700-1759,1828.0,0.0,0,0,160
555447,YUM,"yuma,az",31,4,20304,OO,N710SK,3060,16218,1621802,...,PHX,,2120.0,0.0,2100-2159,2203.0,0.0,0,0,160


In [14]:
# Create a destination dataframe by merging the flights with destination codes
dest_join_df = pd.merge(dest_code_df,flights_2019, on='DEST', how ='inner')
dest_join_df

,DEST,City,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST_AIRPORT_SEQ_ID,dest_city,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,2,20304,OO,N910EV,7363,13487,1348702,...,1014106,,1220.0,1.0,1100-1159,1343.0,1.0,0,0,257
1,ABR,"aberdeen,sd",1,2,20304,OO,N439SW,7419,13487,1348702,...,1014106,,2010.0,0.0,2000-2059,2139.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,3,20304,OO,N498CA,7363,13487,1348702,...,1014106,,1123.0,0.0,1100-1159,1230.0,0.0,0,0,257
3,ABR,"aberdeen,sd",2,3,20304,OO,N935SW,7419,13487,1348702,...,1014106,,2226.0,0.0,2200-2259,2342.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,4,20304,OO,N8965E,7363,13487,1348702,...,1014106,,1120.0,0.0,1100-1159,1224.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555445,YUM,"yuma,az",30,3,20304,OO,N746SK,2988,14107,1410702,...,1621802,,1610.0,0.0,1600-1659,1702.0,0.0,0,0,160
555446,YUM,"yuma,az",31,4,20304,OO,N713SK,2978,14107,1410702,...,1621802,,936.0,0.0,0900-0959,1048.0,0.0,0,0,160
555447,YUM,"yuma,az",31,4,20304,OO,N710SK,2985,14107,1410702,...,1621802,,2254.0,0.0,2200-2259,2347.0,0.0,0,0,160
555448,YUM,"yuma,az",31,4,20304,OO,N713SK,2988,14107,1410702,...,1621802,,1612.0,0.0,1600-1659,1655.0,0.0,0,0,160


In [15]:
# Change city column to Destination City
dest_join_df = dest_join_df.rename(columns={'City':'DEST_CITY'})
dest_join_df

,DEST,DEST_CITY,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST_AIRPORT_SEQ_ID,dest_city,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,2,20304,OO,N910EV,7363,13487,1348702,...,1014106,,1220.0,1.0,1100-1159,1343.0,1.0,0,0,257
1,ABR,"aberdeen,sd",1,2,20304,OO,N439SW,7419,13487,1348702,...,1014106,,2010.0,0.0,2000-2059,2139.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,3,20304,OO,N498CA,7363,13487,1348702,...,1014106,,1123.0,0.0,1100-1159,1230.0,0.0,0,0,257
3,ABR,"aberdeen,sd",2,3,20304,OO,N935SW,7419,13487,1348702,...,1014106,,2226.0,0.0,2200-2259,2342.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,4,20304,OO,N8965E,7363,13487,1348702,...,1014106,,1120.0,0.0,1100-1159,1224.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555445,YUM,"yuma,az",30,3,20304,OO,N746SK,2988,14107,1410702,...,1621802,,1610.0,0.0,1600-1659,1702.0,0.0,0,0,160
555446,YUM,"yuma,az",31,4,20304,OO,N713SK,2978,14107,1410702,...,1621802,,936.0,0.0,0900-0959,1048.0,0.0,0,0,160
555447,YUM,"yuma,az",31,4,20304,OO,N710SK,2985,14107,1410702,...,1621802,,2254.0,0.0,2200-2259,2347.0,0.0,0,0,160
555448,YUM,"yuma,az",31,4,20304,OO,N713SK,2988,14107,1410702,...,1621802,,1612.0,0.0,1600-1659,1655.0,0.0,0,0,160


In [16]:
# Merge the origin and destination dataframes to prepare for the API requests
complete_flight_df = pd.merge(origin_join_df, dest_join_df)
complete_flight_df.head(50)

,ORIGIN,ORIGIN_CITY,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,dest_city,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,DEST_CITY
0,ABR,"aberdeen,sd",1,2,20304,OO,N910EV,7363,10141,1014106,...,,1411.0,1.0,1300-1359,1521.0,1.0,0,0,257,"minneapolis,mn"
1,ABR,"aberdeen,sd",1,2,20304,OO,N432SW,7365,10141,1014106,...,,642.0,0.0,0600-0659,800.0,0.0,0,0,257,"minneapolis,mn"
2,ABR,"aberdeen,sd",2,3,20304,OO,N498CA,7363,10141,1014106,...,,1310.0,0.0,1300-1359,1424.0,0.0,0,0,257,"minneapolis,mn"
3,ABR,"aberdeen,sd",2,3,20304,OO,N439SW,7365,10141,1014106,...,,502.0,0.0,0001-0559,622.0,0.0,0,0,257,"minneapolis,mn"
4,ABR,"aberdeen,sd",3,4,20304,OO,N8965E,7363,10141,1014106,...,,1307.0,0.0,1300-1359,1411.0,0.0,0,0,257,"minneapolis,mn"
5,ABR,"aberdeen,sd",3,4,20304,OO,N935SW,7365,10141,1014106,...,,506.0,0.0,0001-0559,621.0,0.0,0,0,257,"minneapolis,mn"
6,ABR,"aberdeen,sd",4,5,20304,OO,N461SW,7363,10141,1014106,...,,1308.0,0.0,1300-1359,1414.0,0.0,0,0,257,"minneapolis,mn"
7,ABR,"aberdeen,sd",4,5,20304,OO,N461SW,7365,10141,1014106,...,,447.0,0.0,0001-0559,603.0,0.0,0,0,257,"minneapolis,mn"
8,ABR,"aberdeen,sd",5,6,20304,OO,N702BR,7363,10141,1014106,...,,1312.0,0.0,1300-1359,1415.0,0.0,0,0,257,"minneapolis,mn"
9,ABR,"aberdeen,sd",5,6,20304,OO,N430SW,7365,10141,1014106,...,,447.0,0.0,0001-0559,547.0,0.0,0,0,257,"minneapolis,mn"


In [52]:
complete_flight_df.to_csv("Database/complete_flight_df.csv", index=False)

In [17]:
complete_flight_df['ORIGIN_CITY'] = complete_flight_df['ORIGIN_CITY'].astype(str)

In [27]:
# Create an empty list for dept airport codes and date
dest_name = []
origin_name = []
date = []

#Create for loop for dates
for i in complete_flight_df['DAY_OF_MONTH']:
    date.append(i)

#Create for loop for departure airport code
for i in complete_flight_df['ORIGIN_CITY']:
    origin_name.append(i)

#Create for loop for arrival airport code
for i in complete_flight_df['DEST_CITY']:
    dest_name.append(i)


#len(dest_name)
#len(origin_name)
#len(date)

545340

In [48]:
for i, j in complete_flight_df[["DAY_OF_MONTH",'DEST_CITY']]:
    PRINT(i, j)

ValueError: too many values to unpack (expected 2)

In [34]:
# combine lists to make a origin and date tuple
origin_tuple = tuple(zip(origin_name, date))
origin_tuple

(('aberdeen,sd', 1),
 ('aberdeen,sd', 1),
 ('aberdeen,sd', 2),
 ('aberdeen,sd', 2),
 ('aberdeen,sd', 3),
 ('aberdeen,sd', 3),
 ('aberdeen,sd', 4),
 ('aberdeen,sd', 4),
 ('aberdeen,sd', 5),
 ('aberdeen,sd', 5),
 ('aberdeen,sd', 6),
 ('aberdeen,sd', 6),
 ('aberdeen,sd', 7),
 ('aberdeen,sd', 7),
 ('aberdeen,sd', 8),
 ('aberdeen,sd', 8),
 ('aberdeen,sd', 9),
 ('aberdeen,sd', 9),
 ('aberdeen,sd', 10),
 ('aberdeen,sd', 10),
 ('aberdeen,sd', 11),
 ('aberdeen,sd', 11),
 ('aberdeen,sd', 12),
 ('aberdeen,sd', 12),
 ('aberdeen,sd', 13),
 ('aberdeen,sd', 13),
 ('aberdeen,sd', 14),
 ('aberdeen,sd', 14),
 ('aberdeen,sd', 15),
 ('aberdeen,sd', 15),
 ('aberdeen,sd', 16),
 ('aberdeen,sd', 16),
 ('aberdeen,sd', 17),
 ('aberdeen,sd', 17),
 ('aberdeen,sd', 18),
 ('aberdeen,sd', 18),
 ('aberdeen,sd', 19),
 ('aberdeen,sd', 19),
 ('aberdeen,sd', 20),
 ('aberdeen,sd', 20),
 ('aberdeen,sd', 21),
 ('aberdeen,sd', 21),
 ('aberdeen,sd', 22),
 ('aberdeen,sd', 22),
 ('aberdeen,sd', 23),
 ('aberdeen,sd', 23),
 ('abe

In [38]:
# combine lists to make a destination and date tuple
dest_tuple = tuple(zip(dest_name, date))
dest_tuple

(('minneapolis,mn', 1),
 ('minneapolis,mn', 1),
 ('minneapolis,mn', 2),
 ('minneapolis,mn', 2),
 ('minneapolis,mn', 3),
 ('minneapolis,mn', 3),
 ('minneapolis,mn', 4),
 ('minneapolis,mn', 4),
 ('minneapolis,mn', 5),
 ('minneapolis,mn', 5),
 ('minneapolis,mn', 6),
 ('minneapolis,mn', 6),
 ('minneapolis,mn', 7),
 ('minneapolis,mn', 7),
 ('minneapolis,mn', 8),
 ('minneapolis,mn', 8),
 ('minneapolis,mn', 9),
 ('minneapolis,mn', 9),
 ('minneapolis,mn', 10),
 ('minneapolis,mn', 10),
 ('minneapolis,mn', 11),
 ('minneapolis,mn', 11),
 ('minneapolis,mn', 12),
 ('minneapolis,mn', 12),
 ('minneapolis,mn', 13),
 ('minneapolis,mn', 13),
 ('minneapolis,mn', 14),
 ('minneapolis,mn', 14),
 ('minneapolis,mn', 15),
 ('minneapolis,mn', 15),
 ('minneapolis,mn', 16),
 ('minneapolis,mn', 16),
 ('minneapolis,mn', 17),
 ('minneapolis,mn', 17),
 ('minneapolis,mn', 18),
 ('minneapolis,mn', 18),
 ('minneapolis,mn', 19),
 ('minneapolis,mn', 19),
 ('minneapolis,mn', 20),
 ('minneapolis,mn', 20),
 ('minneapolis,mn'

In [43]:
for date in dest_tuple[date]:
    print(date)

TypeError: tuple indices must be integers or slices, not tuple

### Create API calls using Open Weather

In [19]:
# Starting URL for API call to World Weather
url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx" + "?key=" + weather_api_key
print(url)

http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=95b1b7beca9a449991c03005202108


In [53]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(dest_name):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    airport_url = url + "&q=" + dest_name + "&format=json"

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(airport_url).json()
        # Parse out the needed data.
        city_lat = city_weather["weather"]["date"]['hourly']
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | m
City not found. Skipping...
Processing Record 2 of Set 1 | i
City not found. Skipping...
Processing Record 3 of Set 1 | n
City not found. Skipping...
Processing Record 4 of Set 1 | n
City not found. Skipping...
Processing Record 5 of Set 1 | e
City not found. Skipping...
Processing Record 6 of Set 1 | a
City not found. Skipping...
Processing Record 7 of Set 1 | p
City not found. Skipping...
Processing Record 8 of Set 1 | o
City not found. Skipping...
Processing Record 9 of Set 1 | l
City not found. Skipping...
Processing Record 10 of Set 1 | i
City not found. Skipping...
Processing Record 11 of Set 1 | s
City not found. Skipping...
Processing Record 12 of Set 1 | ,
City not found. Skipping...
Processing Record 13 of Set 1 | m
City not found. Skipping...
Processing Record 14 of Set 1 | n
City not found. Skipping...
-----------------------------
Data Retrieval Complete      
---------------------

In [41]:
# Loop through the list.
for dest_name, date in dest_tuple:

    # Create endpoint URL with each city.
    airport_url = url + "&q=" + dest_name + "&format=json" + "&date=2019-01-" + date
    
    print(airport_url)

TypeError: can only concatenate str (not "int") to str

In [20]:
# Import Historical API Package
from wwo_hist import retrieve_hist_data


In [56]:
start_date = '01-JAN-2019'
end_date = '31-JAN-2019'
location_list = ["dallas,tx","austin,tx","houston,tx"]
frequency = 1

In [62]:
hist_weather_data = retrieve_hist_data(weather_api_key,
                                   location_list,
                                   start_date,
                                   end_date,
                                   frequency,
                                   location_label = False,
                                   export_csv = True,
                                   store_df = True)



Retrieving weather data for dallas,tx


Currently retrieving data for dallas,tx: from 2019-01-01 to 2019-01-31
Time elapsed (hh:mm:ss.ms) 0:00:00.941671


export dallas,tx completed!




Retrieving weather data for austin,tx


Currently retrieving data for austin,tx: from 2019-01-01 to 2019-01-31
Time elapsed (hh:mm:ss.ms) 0:00:00.995655


export austin,tx completed!




Retrieving weather data for houston,tx


Currently retrieving data for houston,tx: from 2019-01-01 to 2019-01-31
Time elapsed (hh:mm:ss.ms) 0:00:01.005567


export houston,tx completed!




AttributeError: module 'pandas' has no attribute 'store_df'

In [80]:
import os, glob


all_files = glob.glob(os.path.join("*.csv"))

all_df = []
for f in all_files:
    df = pd.read_csv(f, sep=',')
    df['file'] = f.split('/')[-1]
    all_df.append(df)
    
merged_df = pd.concat(all_df, ignore_index=True, sort=True)

In [81]:
merged_df

,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,date_time,file,humidity,location,...,pressure,sunHour,sunrise,sunset,tempC,totalSnow_cm,uvIndex,visibility,winddirDegree,windspeedKmph
0,3,5,6,5,12,0,2019-01-01 00:00:00,"dallas,tx.csv",82,"dallas,tx",...,1019,5.0,08:29 AM,06:32 PM,6,0.0,2,10,349,10
1,3,4,6,4,16,6,2019-01-01 01:00:00,"dallas,tx.csv",82,"dallas,tx",...,1020,5.0,08:29 AM,06:32 PM,5,0.0,2,10,274,12
2,3,2,5,2,21,12,2019-01-01 02:00:00,"dallas,tx.csv",83,"dallas,tx",...,1021,5.0,08:29 AM,06:32 PM,4,0.0,2,10,199,13
3,2,1,5,1,26,18,2019-01-01 03:00:00,"dallas,tx.csv",83,"dallas,tx",...,1021,5.0,08:29 AM,06:32 PM,4,0.0,2,10,124,15
4,2,1,4,1,28,33,2019-01-01 04:00:00,"dallas,tx.csv",81,"dallas,tx",...,1022,5.0,08:29 AM,06:32 PM,3,0.0,2,10,84,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,10,14,14,14,28,100,2019-01-31 19:00:00,"houston,tx.csv",75,"houston,tx",...,1021,5.2,08:12 AM,06:58 PM,14,0.0,4,9,117,14
2228,11,14,14,14,27,100,2019-01-31 20:00:00,"houston,tx.csv",79,"houston,tx",...,1021,5.2,08:12 AM,06:58 PM,13,0.0,4,8,118,13
2229,11,14,14,14,26,100,2019-01-31 21:00:00,"houston,tx.csv",83,"houston,tx",...,1020,5.2,08:12 AM,06:58 PM,13,0.0,4,8,118,11
2230,12,14,14,14,23,99,2019-01-31 22:00:00,"houston,tx.csv",85,"houston,tx",...,1020,5.2,08:12 AM,06:58 PM,13,0.0,4,8,117,10


In [69]:
dallas,tx

NameError: name 'dallas' is not defined